# Associations between molecular and environmental changes along the proximal-to-distal axis of the colon
## Abstract
### Objective
The two-colon paradigm, differentiating colorectal cancers according to their location relative to the splenic flexure, contributed to improved prognosis and treatment. Recent studies challenged this division by proposing a continuum model where molecular properties follow a continuous trend along the colon. We address the question of which model describes colorectal cancer properties better by comparing their performance in describing such properties.
### Design
We used linear, polynomial, and sigmoid models to fit the spatial dependency of molecular and environmental tumour features along the proximal-to-distal axis of the colon leveraging a cohort of 522 colorectal cancer patients from TCGA and quantified the tumour metabolome in a novel cohort of 27 patients. Subsequently, we developed a network analysis approach to identify closely interacting genes with experimentally quantified metabolites to study the interplay between molecular and environmental features of tumours.
### Results
Sigmoid behaviours are exclusively observed for features related to molecular properties of tumours. At the same time, continuous models approximate better properties of the tumour environment and specifically alterations of genes responding to or modulating this environment. As this suggests an environmental impact on selective constraints related to the changes of alteration profiles along the colon, we aim to understand better the interplay of components of the tumour ecosystem. We show that genes with continuous transcriptional profiles interact with metabolites associated with carcinogenesis, which in turn tend to follow continuous trends correlated with bacterial abundance (which show continuous behaviour themselves). Overall, these results suggest that gradual changes in environmental factors along the colon contribute to shaping the selective forces driving carcinogenesis.
### Conclusion
By capturing the biology of tumours as systems interacting with their local environment, our results demonstrate the clinical relevance of increasing the resolution of tumour localisation.

### About this notebook
This notebook allow to reproduce all the results and figures presented in [CITAZIONE] 

# LOADING LIBRARIES

In [240]:
%matplotlib inline
# Import custom libraries
import sys
sys.path.append("../../../../git/profiler/lib") # Path to the profile_analysis_class.py lib
# sys.path.append("../../../git/profiler/lib") # Path to the profile_analysis_class.py lib
sys.path.append("../../../../git/network_analysis/lib/") # Path to the network_search_class.py lib
# sys.path.append("../../../git/network_analysis/lib/") # Path to the network_search_class.py lib
from profile_analysis_class import ProfileAnalysis
from network_search_class import NetworkSearchAnalysis

#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from supervenn import supervenn
import mygene
from joblib import Parallel, delayed
from statsmodels.stats.multitest import fdrcorrection
import os
import statistics
import math
from scipy import stats
from scipy.stats import beta, norm, fisher_exact, mannwhitneyu, chi2_contingency, ranksums
import scipy.stats
import collections

# Defining left and right sections
left_sections = ['Descending colon','Sigmoid colon','Rectosigmoid junction','Rectum, NOS']
right_sections = ['Cecum','Ascending colon','Hepatic flexure of colon','Transverse colon']

# Create mygene class for info on gene ids
mg = mygene.MyGeneInfo()

# Set graphical options
plt.style.use('../../assets/styles/plotting_style.mplstyle') # Path to the matplotlib style sheet

## Utility Functions

In [241]:
# Remove chemicals with gene-chemical interactions above threshold
# Get all the interaction types present in the CTD database
def ctd_filtering(group,threshold):
    interaction_types = []
    ctd_data_filtered = pd.DataFrame()
    for interaction in group['Interaction Actions']:
        interaction_types = interaction_types + interaction.split('|')
    if len(group) <= threshold:
        ctd_data_filtered = pd.concat([ctd_data_filtered,group])
    return ctd_data_filtered, set(interaction_types), interaction_types

# TRANSCRIPTOME PROFILING

In [242]:
# Create workflow class, specifying the path to the SETTINGS.ini file
pa_transcriptome = ProfileAnalysis('../../../../docker/analysis/transcriptome')

Project has been created!


/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:154: UserWarning: Folder input_data already exist...skipping
  warnings.warn("Folder input_data already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:160: UserWarning: Folder data_raw already exist...skipping
  warnings.warn("Folder data_raw already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:166: UserWarning: Folder data_clinical already exist...skipping
  warnings.warn("Folder data_clinical already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:172: UserWarning: Folder sample_by_section already exist...skipping
  warnings.warn("Folder sample_by_section alr

## Assign each sample in clinical data file to a colon section

In [243]:
pa_transcriptome.create_samples_to_sections_table()

## Calculate median value and median average variation for each colon section

In [244]:
medians_tr, mad_tr = pa_transcriptome.median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


## GTEX filtering

In [245]:
# The following code calculate the right/left ratio for GTEX and TCGA expression data
# and use this ratio to estimate threshold of high and low expression ratio

def dLR(data, left_sec = 'Sigmoid colon', right_sec = 'Transverse colon'):
    # Create a dataframe with the selected section as right left sections
    dLR = pd.DataFrame(index=data.index)
    dLR['left']=data[{left_sec}]
    dLR['right']=data[{right_sec}]
    dLR['tcga_r/l']=dLR['right']/dLR['left']
    return dLR

def estimateT(median_exp, z=[2,1.5,1,0.5]):
    # Estimate the z-score threshold for high\low expression ratio by fitting a gaussian on GTEX
    mean=np.mean(median_exp)
    std=np.std(median_exp)
    ts = {}
    for value in z:
        hit=(value*std)+mean
        lowt=mean-(value*std)
        ts[value] = tuple([hit,lowt])
    return ts

# To save execution time the GTEX file have already been generated
# To regenerate the file just delete it ('../docker/analysis/gtex/gtex_right_left_diff.csv')
if not os.path.isfile('../../../../docker/analysis/gtex/gtex_right_left_diff.csv'):
    # Load GTEX data
    gtex=pd.read_csv('../../../../docker/analysis/gtex/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gzip', compression='gzip')
    gtex.set_index('Name', inplace=True)
    # Load GTEX clinical data
    gtex_clinical=pd.read_csv('../../../../docker/analysis/gtex/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt', sep='\t')
    gtex_clinical.set_index('SAMPID', inplace=True)
    # Discard samples for which we don't have data
    mask = [index for index in gtex_clinical.index if index in gtex.columns]
    gtex_clinical = gtex_clinical.loc[mask]
    # Classify samples as left or right
    gtex_samples2sections={}
    gtex_samples2sections['left']=gtex_clinical[gtex_clinical['SMTSD']=='Colon - Sigmoid'].index
    gtex_samples2sections['right']=gtex_clinical[gtex_clinical['SMTSD']=='Colon - Transverse'].index
    # Calculate left and right median expression for each gene
    gtex_data=pd.DataFrame()
    for index, row in gtex.iterrows():
        gtex_data.loc[index, 'left']=statistics.median(row[gtex_samples2sections['left']])
        gtex_data.loc[index, 'right']=statistics.median(row[gtex_samples2sections['right']])
    # Remove the .XX part of ensmbl id
    gtex_lr=pd.DataFrame(columns=gtex_data.columns)
    for index in gtex_data.index:
        gtex_lr.loc[index.split('.')[0]]=gtex_data.loc[index]
    # calculate GTEX RIGHT/LEFT ratio for each gene
    gtex_lr['gtex_r/l'] = (gtex_lr['right'])/(gtex_lr['left'])
    # Drop na (happens when left TPM counts are 0)
    gtex_lr.dropna(inplace=True)
    # Save file for future
    gtex_lr.index.name = 'ensmbl'
    gtex_lr.to_csv('../../../../docker/analysis/gtex/gtex_right_left_diff.csv')
else:
    gtex_lr = pd.read_csv('../../../../docker/analysis/gtex/gtex_right_left_diff.csv')
    gtex_lr.set_index('ensmbl', inplace=True)

# calculate right/left ratio in TCGA data
medians_lr = dLR(medians_tr)

# Merge GTEX and TCGA, drop inf and na values
gtex_tcga = pd.concat([medians_lr['tcga_r/l'],gtex_lr['gtex_l-r']], axis=1)
gtex_tcga.replace([np.inf, -np.inf], np.nan, inplace=True)
gtex_tcga.dropna(inplace=True)

# Estimate the z-score threshold for high\low expression ratio by fitting a gaussian on TCGA data
ts_tcga = estimateT(gtex_tcga['tcga_r/l'])

# Select continuous genes inside the top left and bottom right panels of previous plot
# they correspond to high/low ratio GTEX vs TCGA and low/high ratio GTEX vs TCGA
ds_tcga=0.5
ds_gtex=1
tcga_std_filter = [feature for feature in gtex_tcga['tcga_r/l'].index
                   if gtex_tcga['tcga_r/l'][feature]>ts_tcga[ds_tcga][0] 
                   or gtex_tcga['tcga_r/l'][feature]<ts_tcga[ds_tcga][1]]
gtex_filter = [feature for feature in tcga_std_filter 
               if gtex_tcga['gtex_l-r'][feature]<ts_tcga[ds_gtex][0] 
               and gtex_tcga['gtex_l-r'][feature]>ts_tcga[ds_gtex][1]]

medians_tr = medians_tr.loc[gtex_filter]

<ipython-input-245-ae584ffe66c4>:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dLR['left']=data[{left_sec}]
<ipython-input-245-ae584ffe66c4>:8: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dLR['right']=data[{right_sec}]


## Fit Observables

In [7]:
scores_tr, poly_obs_scores_tr, sig_obs_scores_tr, poly_models_tr, sig_models_tr = pa_transcriptome.fit_data(medians_tr, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit randomly permutated data

In [8]:
poly_perm_scores_tr, sig_perm_scores_tr, sig_perm_models_tr = pa_transcriptome.fit_random_data(medians_tr, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Plot models performance

In [9]:
models_pvalue_tr, scores_table=pa_transcriptome.plot_gof(poly_obs_scores_tr, sig_obs_scores_tr, poly_perm_scores_tr, sig_perm_scores_tr, dist_perm=True)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:327: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:333: RuntimeWarning: invalid value encountered in accumulate
  pvals_corrected = np.minimum.accumulate(pvals_corrected_raw[::-1])[::-1]
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:335: RuntimeWarning

## Determine which models performed better on observed than randomly generated data

In [10]:
poly_obs_scores_tr = pa_transcriptome.select_significant_models(models_pvalue_tr, poly_obs_scores_tr)

## Generate background distributions datasets

In [11]:
backgrounds_tr = pa_transcriptome.assemble_backgrounds(poly_obs_scores_tr, sig_perm_scores_tr, poly_perm_scores_tr)

## Calculate p and q values

In [12]:
p_value_tables_tr = pa_transcriptome.get_p_values(poly_obs_scores_tr, backgrounds_tr)

In [13]:
q_value_tables_tr = pa_transcriptome.get_q_values(p_value_tables_tr, poly_obs_scores_tr)

/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:327: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:333: RuntimeWarning: invalid value encountered in accumulate
  pvals_corrected = np.minimum.accumulate(pvals_corrected_raw[::-1])[::-1]
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:335: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [14]:
backgrounds_tr_str = {}
for key in backgrounds_tr.keys():
    backgrounds_tr_str[str(key)] = backgrounds_tr[key]

In [15]:
backgrounds_all_scores_dist = {}
for key in backgrounds_tr_str:
    temp = []
    for column in backgrounds_tr_str[key]:
        temp = temp + list(backgrounds_tr_str[key][column])
    backgrounds_all_scores_dist[str(key)] = temp

In [16]:
classification_tr = pa_transcriptome.classify_genes(poly_obs_scores_tr, q_value_tables_tr, 0.2, 0.1)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:782: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_evaluate.dropna(subset=polynomial_columns, how='all', inplace=True)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:791: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_evaluate['continuous'] = only_continuous.iloc[:,0]


In [17]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_tr.items() ])).to_csv(f'{pa_transcriptome.output}/classification_tr.csv')

In [18]:
for key in classification_tr:
    print(key)
    print(len(classification_tr[key]))

sigmoid
259
discarded
44
continuous
846


## SF 1A

In [19]:
gene_list_tr, section_l_tr = pa_transcriptome.strict_sig_list(classification_tr['sigmoid'], sig_models_tr, plot_dist = True)

<lambdifygenerated-1>:3: RuntimeWarning: overflow encountered in exp
  return (c*k*exp(-k*(x - x0))/(1 + exp(-k*(x - x0)))**2)
<lambdifygenerated-1>:3: RuntimeWarning: invalid value encountered in double_scalars
  return (c*k*exp(-k*(x - x0))/(1 + exp(-k*(x - x0)))**2)
<lambdifygenerated-1>:3: RuntimeWarning: overflow encountered in double_scalars
  return (c*k*exp(-k*(x - x0))/(1 + exp(-k*(x - x0)))**2)


## Figure 1D

In [20]:
pa_transcriptome.plot_fitting_bars(classification_tr['continuous'],
                                    ['ENSG00000134982', 'ENSG00000118705'],
                                    medians_tr,
                                    mad_tr,
                                    poly_models_tr, sig_models_tr,
                                    'continuum',
                                    title=False,
                                    save_as='Figure_1D.svg')

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:1062: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


# CTD enrichment for each chemical

In [21]:
#Preparing ctd dataset, for all chemicals (ctd_data), and for those associated with colorectal neoplasms (ctd_data_colorectalneoplasm_ds)
ctd_data = pd.read_csv('../../../../docker/analysis/ctd/CTD_chem_gene_ixns.csv', skiprows=27)
ctd_data = ctd_data[ctd_data['Organism']=='Homo sapiens']
ctd_data = ctd_data[['# ChemicalName','ChemicalID','CasRN','GeneSymbol','GeneID','Interaction','InteractionActions']]
ctd_data.columns = ['Chemical Name','Chemical ID','CAS RN','Gene Symbol','Gene ID','Interaction','Interaction Actions']

ctd_data_colorectalneoplasm_ds = pd.read_csv('../../../../docker/analysis/ctd/CTD_D015179_ixns_20230518052747.csv')
ctd_data_colorectalneoplasm_ds = ctd_data_colorectalneoplasm_ds[ctd_data_colorectalneoplasm_ds['Reference Count']>=10]
ctd_data_colorectalneoplasm_ds = ctd_data_colorectalneoplasm_ds[['Chemical Name','Chemical ID','CAS RN','Gene Symbol','Gene ID','Interaction','Interaction Actions']]

In [22]:
# Determine continuous and not continuous genes
cont_gtex_filtered = classification_tr['continuous']
cont = set(cont_gtex_filtered)
no_cont = set(medians_tr.index)
no_cont = no_cont.symmetric_difference(cont)

In [23]:
# Group CTD data by chemical
ctd_by_chemicalid = ctd_data.groupby('Chemical ID')

In [24]:
# Remove chemicals with gene-chemical interactions above threshold (10000)
# Get all the interaction types present in the CTD database
threshold = 10000
calc_res = Parallel(n_jobs=20)(delayed(ctd_filtering)(group, threshold) for name, group in ctd_by_chemicalid)

In [25]:
# unpack interactions result after paralle calculation
interaction_types_unique = []
interaction_types_full = []
ctd_database_df = pd.DataFrame()
for result in calc_res:
    interaction_types_unique = interaction_types_unique + list(result[1])
    interaction_types_full = interaction_types_full + list(result[2])
    ctd_database_df = pd.concat([ctd_database_df, result[0]])
interaction_types_unique = set(interaction_types_unique)

In [26]:
# Convert CTD gene symbols to ensmbl id
query = mg.querymany(set(ctd_database_df['Gene Symbol']),
                    scopes='symbol',
                    species=9606,
                    fields='ensembl',
                    as_dataframe=True)
ctd_database_df['ensembl_id']=np.nan
query.dropna(subset=['ensembl.gene'], inplace=True)
for gene in query.index:
    if isinstance(query.loc[gene]['ensembl.gene'], str):
        ens = query.loc[gene]['ensembl.gene']
    else:
        ens = query.loc[gene]['ensembl.gene'][0]
    ctd_database_df.loc[(ctd_database_df['Gene Symbol']==gene), 'ensembl_id'] = ens
ctd_database_df.dropna(subset=['ensembl_id'], inplace=True)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-25994...done.
Finished.
2742 input query terms found dup hits:
	[('ZKSCAN8P1', 2), ('RPL23AP7', 2), ('PI4KAP1', 2), ('MT1JP', 2), ('EIF1P7', 2), ('UBE2E4P', 2), ('M
995 input query terms found no hit:
	['nan', 'ZFP346', 'GH', 'GRAMD2', 'FAM84B', 'PHLPP', 'C1ORF68', 'STCH', 'PIAS', 

In [27]:
#create a chemical-interactions dictionary
compounds_interactions = {}
ctd_by_chemicalid = ctd_database_df.groupby('Chemical Name')
for name, group in ctd_by_chemicalid:
    compounds_interactions[name] = group

In [28]:
# Test enrichment for each chemical
chemical_enrichment = pd.DataFrame()
i = 0
for key in compounds_interactions:
    genes_in_ctd = set(compounds_interactions[key]['ensembl_id'])
    #genes continuous/ctd
    cont_ctd = len(cont.intersection(genes_in_ctd))
    #genes sigmoid/ctd
    no_cont_ctd = len(no_cont.intersection(genes_in_ctd))
    #genes continuous/no_ctd
    cont_no_ctd = len(cont) - cont_ctd
    #genes sigmoid/no_ctd
    no_cont_no_ctd = len(no_cont) - no_cont_ctd
    #calculate contingency table
    table=[[cont_ctd,cont_no_ctd],[no_cont_ctd,no_cont_no_ctd]]
    stat = []
    try:
        stat = chi2_contingency(table)
    except:
        i = i + 1
    if stat:
        chemical_enrichment.loc[key,'chi2'] = stat[0]
        chemical_enrichment.loc[key,'p-value'] = stat[1]
        chemical_enrichment.loc[key,'table'] = [str(table)]
print(f'{i} compound were discarded because the table of expected frequency contains zeroes')
chemical_enrichment['q-value'] = fdrcorrection(chemical_enrichment['p-value'])[1]

4869 compound were discarded because the table of expected frequency contains zeroes


In [29]:
chemical_enrichment[chemical_enrichment['q-value']<=0.2]

,chi2,p-value,table,q-value
1-Methyl-3-isobutylxanthine,31.531570,1.962240e-08,"[[[113, 733], [253, 3219]]]",0.000019
3-((6-(2-methoxyphenyl)pyrimidin-4-yl)amino)phenyl)methane sulfonamide,17.934398,2.286510e-05,"[[[40, 806], [72, 3400]]]",0.006359
3-iodothyronamine,11.719643,6.184380e-04,"[[[4, 842], [0, 3472]]]",0.108928
"7,8-Dihydro-7,8-dihydroxybenzo(a)pyrene 9,10-oxide",14.802636,1.193685e-04,"[[[102, 744], [272, 3200]]]",0.026281
Acetaminophen,12.820968,3.427563e-04,"[[[198, 648], [623, 2849]]]",0.062453
Antirheumatic Agents,20.550728,5.807169e-06,"[[[142, 704], [383, 3089]]]",0.002192
Cadmium,27.191812,1.842373e-07,"[[[134, 712], [332, 3140]]]",0.000108
Cadmium Chloride,18.796367,1.454439e-05,"[[[131, 715], [353, 3119]]]",0.004521
Caffeine,17.196265,3.370984e-05,"[[[92, 754], [230, 3242]]]",0.008482
Copper Sulfate,17.324998,3.150154e-05,"[[[177, 669], [520, 2952]]]",0.008323


In [30]:
# Testing if metabolites found to be associated with CRC are higer than expected by chance
i=0
j=0
y=0
z=0
chemical_found = list(chemical_enrichment[chemical_enrichment['q-value']<=0.2].index)
for x in chemical_found:
    if x in list(ctd_data_colorectalneoplasm_ds['Chemical Name']):
        i=i+1
    else:
        j=j+1
for x in ctd_data['Chemical Name'].unique():
    if x not in chemical_found:
        if x in list(ctd_data_colorectalneoplasm_ds['Chemical Name'].unique()):
            y=y+1
        else:
            z=z+1

In [31]:
chi2_contingency([[i,y],[j,z]])

Chi2ContingencyResult(statistic=44.49552600214488, pvalue=2.549391367266909e-11, dof=1, expected_freq=array([[7.58956086e-01, 2.12241044e+02],
       [3.62410439e+01, 1.01347590e+04]]))

# METHYLOME PROFILING

In [34]:
# Create workflow class, specifying the path to the SETTINGS.ini file
pa_methylome = ProfileAnalysis('../../../../docker/analysis/methylome')

Project has been created!


## Assign each sample in clinical data file to a colon section

In [35]:
pa_methylome.create_samples_to_sections_table()

## Calculate median value for each colon section

In [36]:
medians_meth, mad_meth = pa_methylome.median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


In [37]:
# Exclude promoters outside +-1sd
mean = np.median(medians_meth.median(axis=1))
std = np.std(medians_meth.median(axis=1))
hit = std + mean
lowt = mean - std
medians_meth = medians_meth.loc[(medians_meth.median(axis=1)<hit) & (medians_meth.median(axis=1)>lowt)]

## SF 2A

In [39]:
# Supplementary Figure 2A, left vs right methylation
left = []
right =  []
for section in left_sections:
    left=left+list(medians_meth[section])
for section in right_sections:
    right=right+list(medians_meth[section])
# Wilcoxon test
stat, p = ranksums(left, right, alternative='less')
p = '{:.2e}'.format(p)
plt.figure(figsize=(8,8))
plt.boxplot([left, right], labels=['Left', 'Right'])
plt.ylabel('Promoter methylation Level')
plt.title(f'Wilcoxon rank-sum test,  p = {p}')
plt.savefig('../../../../docker/analysis/methylome/figures/SF2A.svg')

## Fit Observables

In [40]:
scores_meth, poly_obs_scores_meth, sig_obs_scores_meth, poly_models_meth, sig_models_meth = pa_methylome.fit_data(medians_meth, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Random permutated data

In [41]:
poly_perm_scores_meth, sig_perm_scores_meth, sig_perm_models_meth=pa_methylome.fit_random_data(medians_meth, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


In [43]:
models_pvalue_meth, scores_table=pa_methylome.plot_gof(poly_obs_scores_meth, sig_obs_scores_meth, poly_perm_scores_meth, sig_perm_scores_meth, dist_perm=True)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:327: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:333: RuntimeWarning: invalid value

In [44]:
poly_obs_scores_meth = pa_methylome.select_significant_models(models_pvalue_meth, poly_obs_scores_meth)

In [45]:
backgrounds_meth = pa_methylome.assemble_backgrounds(poly_obs_scores_meth, sig_perm_scores_meth, poly_perm_scores_meth)

## Compare obrservable vs permutated data

In [46]:
p_value_tables_meth = pa_methylome.get_p_values(poly_obs_scores_meth, backgrounds_meth)

In [47]:
q_value_tables_meth = pa_methylome.get_q_values(p_value_tables_meth, poly_obs_scores_meth)

/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:327: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:333: RuntimeWarning: invalid value encountered in accumulate
  pvals_corrected = np.minimum.accumulate(pvals_corrected_raw[::-1])[::-1]
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:335: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [48]:
models_pvalue_meth

{'sigmoidal': [15250937423.0, 0.0, 0.6784454193687428],
 1: [22609570691.0, 0.0, 0.46323705510079105],
 2: [20668229333.0, 0.0, 0.6815298157198942],
 3: [18178819894.0, 0.0, 0.8271700644183259]}

In [49]:
backgrounds_meth_str = {}
for key in backgrounds_meth.keys():
    backgrounds_meth_str[str(key)] = backgrounds_meth[key]

In [50]:
backgrounds_all_scores_dist = {}
for key in backgrounds_meth:
    temp = []
    for column in backgrounds_meth[key]:
        temp = temp + list(backgrounds_meth[key][column])
    backgrounds_all_scores_dist[str(key)] = temp

In [51]:
classification_meth = pa_methylome.classify_genes(poly_obs_scores_meth, q_value_tables_meth, 0.2, 0.1)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:782: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_evaluate.dropna(subset=polynomial_columns, how='all', inplace=True)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:791: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_evaluate['continuous'] = only_continuous.iloc[:,0]


In [52]:
for key in classification_meth:
    print(key)
    print(len(classification_meth[key]))

sigmoid
1435
discarded
120
continuous
7976


In [53]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_meth.items() ])).to_csv(f'{pa_methylome.output}/classification_meth.csv')

## Cluster genes

## SF 1B

In [54]:
gene_list_meth, section_l_meth = pa_methylome.strict_sig_list(classification_meth['sigmoid'], sig_models_meth, plot_dist = True)

# Overlap with methyldriver

In [55]:
all_r_genes=pd.read_csv('../../../../docker/analysis/methyldriver/methyldriver_all_genes.csv')
all_r_genes.set_index('genes', inplace=True)
for index, row in medians_meth.iterrows():
    medians_meth.loc[index, 'gene'] = index.split('_')[0]
medians_meth.set_index('gene', inplace=True)

In [56]:
total_genes = all_r_genes.index.intersection(medians_meth.index)

In [57]:
diff_methylated = pd.read_csv('../../../../docker/analysis/methyldriver/COAD_hyper_hypo_methylated_genes.csv')
diff_methylated.set_index('gene_symbol', inplace=True)
diff_methylated = diff_methylated.index.intersection(total_genes)

In [58]:
diff_methylated

Index(['FAM200A', 'ZNF655', 'MLH1', 'ITPKB', 'ZNF345', 'HSPA1A', 'CCT6B',
       'QKI', 'HSPA1L', 'ADHFE1',
       ...
       'PTMA', 'OSGIN2', 'IMMT', 'COG5', 'KLRG1', 'BEST3', 'DLD', 'LARP4',
       'NFYA', 'MMAB'],
      dtype='object', length=418)

In [59]:
trend = list(set([i.split('_')[0] for i in classification_meth['continuous']]))
trend = trend +list(set([i.split('_')[0] for i in classification_meth['sigmoid']]))
trend = set(trend)
trend = trend.intersection(total_genes)
len(trend)

7158

In [60]:
no_trend = [gene for gene in total_genes if gene not in trend]
len(no_trend)

4556

In [61]:
no_hypo = [gene for gene in total_genes if gene not in diff_methylated]
len(no_hypo)

11296

In [62]:
t_h = trend.intersection(diff_methylated)
len(t_h)

292

In [63]:
nt_h = set(no_trend).intersection(diff_methylated)
len(nt_h)

126

In [64]:
t_nh = trend.intersection(no_hypo)
len(t_nh)

6866

In [65]:
nt_nh = set(no_trend).intersection(no_hypo)
len(nt_nh)

4430

In [66]:
import numpy as np
table = np.array([[len(t_h), len(nt_h)],[len(t_nh), len(nt_nh)]])
oddsr, p = fisher_exact(table, alternative='two-sided')
p

0.000184969651841633

# MUTATIONS PROFILING

In [67]:
# Create workflow class, specifying the path to the SETTINGS.ini file
pa_mutations = ProfileAnalysis('../../../../docker/analysis/mutations')

Project has been created!


/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:154: UserWarning: Folder input_data already exist...skipping
  warnings.warn("Folder input_data already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:160: UserWarning: Folder data_raw already exist...skipping
  warnings.warn("Folder data_raw already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:166: UserWarning: Folder data_clinical already exist...skipping
  warnings.warn("Folder data_clinical already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:172: UserWarning: Folder sample_by_section already exist...skipping
  warnings.warn("Folder sample_by_section alr

## Calculate median value for each colon section

In [68]:
medians_mut, mad_mut = pa_mutations.median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


In [69]:
medians_mut.loc[medians_mut.isin([0]).sum(axis=1)<1]

,Cecum,Ascending_Colon,Hepatic_Flexure,Transverse_Colon,Descending_Colon,Sigmoid_Colon,Rectosigmoid_Junction,Rectum
ensmbl_id,,,,,,,,
A2M,6.024096,8.000000,5.555556,12.903226,11.111111,2.752294,6.976744,6.849315
AASS,7.228916,4.000000,11.111111,6.451613,11.111111,0.917431,2.325581,6.849315
ABCA3,12.048193,14.666667,11.111111,9.677419,5.555556,0.917431,2.325581,4.109589
ABCA4,10.843373,8.000000,22.222222,12.903226,5.555556,7.339450,2.325581,5.479452
ABCB5,4.819277,6.666667,5.555556,16.129032,5.555556,2.752294,6.976744,6.849315
...,...,...,...,...,...,...,...,...
ZNF646,13.253012,9.333333,16.666667,6.451613,5.555556,6.422018,2.325581,6.849315
ZNF687,4.819277,4.000000,11.111111,12.903226,5.555556,1.834862,2.325581,4.109589
ZNF804A,7.228916,10.666667,22.222222,12.903226,5.555556,5.504587,2.325581,6.849315


In [70]:
medians_mut.columns = ['Cecum', 'Ascending colon', 'Hepatic flexure of colon', 'Transverse colon', 'Descending colon', 'Sigmoid colon', 'Rectosigmoid junction', 'Rectum, NOS']

# SF 2B

In [71]:
left = []
right =  []
for section in left_sections:
    left=left+list(medians_mut[section])
for section in right_sections:
    right=right+list(medians_mut[section])
# Wilcoxon test
stat, p = stats.ranksums(left, right, alternative='less')
p = '{:.2e}'.format(p)
plt.figure(figsize=(8,8))
plt.boxplot([left, right], labels=['Left', 'Right'])
plt.ylabel('Mutation prevalence')
plt.title(f'Wilcoxon rank-sum test,  p = {p}')
plt.savefig('../../../../docker/analysis/mutations/figures/SF2B.svg')

## Fit Observables

In [72]:
scores_mut, poly_obs_scores_mut, sig_obs_scores_mut, poly_models_mut, sig_models_mut = pa_mutations.fit_data(medians_mut, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Random permutated data

In [73]:
poly_perm_scores_mut, sig_perm_scores_mut, sig_perm_models_mut=pa_mutations.fit_random_data(medians_mut, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Compare obrservable vs permutated data

In [74]:
models_pvalue_mut, scores_table=pa_mutations.plot_gof(poly_obs_scores_mut, sig_obs_scores_mut, poly_perm_scores_mut, sig_perm_scores_mut, dist_perm=True)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/

In [75]:
poly_obs_scores_mut = pa_mutations.select_significant_models(models_pvalue_mut, poly_obs_scores_mut)

In [76]:
backgrounds_mut = pa_mutations.assemble_backgrounds(poly_obs_scores_mut, sig_perm_scores_mut, poly_perm_scores_mut)

In [77]:
p_value_tables_mut = pa_mutations.get_p_values(poly_obs_scores_mut, backgrounds_mut)

In [78]:
q_value_tables_mut = pa_mutations.get_q_values(p_value_tables_mut, poly_obs_scores_mut)

In [79]:
backgrounds_mut_str = {}
for key in backgrounds_mut.keys():
    backgrounds_mut_str[str(key)] = backgrounds_mut[key]

In [80]:
classification_mut = pa_mutations.classify_genes(poly_obs_scores_mut, q_value_tables_mut, 0.2,0.1)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:782: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_evaluate.dropna(subset=polynomial_columns, how='all', inplace=True)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:791: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_evaluate['continuous'] = only_continuous.iloc[:,0]


In [81]:
for key in classification_mut:
    print(key)
    print(len(classification_mut[key]))

sigmoid
202
discarded
42
continuous
306


In [82]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_mut.items() ])).to_csv(f'{pa_mutations.output}/classification_mut.csv')

## SF 1C

In [83]:
gene_list_mut, section_l_mut = pa_mutations.strict_sig_list(classification_mut['sigmoid'], sig_models_mut, plot_dist = True)

<lambdifygenerated-3>:3: RuntimeWarning: overflow encountered in double_scalars
  return (c*k*exp(-k*(x - x0))/(1 + exp(-k*(x - x0)))**2)


# CMS PROFILING

In [84]:
# Create workflow class, specifying the path to the SETTINGS.ini file
pa_cms = ProfileAnalysis('../../../../docker/analysis/cms')

Project has been created!


/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:154: UserWarning: Folder input_data already exist...skipping
  warnings.warn("Folder input_data already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:160: UserWarning: Folder data_raw already exist...skipping
  warnings.warn("Folder data_raw already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:166: UserWarning: Folder data_clinical already exist...skipping
  warnings.warn("Folder data_clinical already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:172: UserWarning: Folder sample_by_section already exist...skipping
  warnings.warn("Folder sample_by_section alr

## Assign each sample in clinical data file to a colon section

In [85]:
pa_cms.create_samples_to_sections_table()

## Calculate median value for each colon section

In [86]:
medians_cms, mad_cms = pa_cms.median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Observables

In [87]:
scores_cms, poly_obs_scores_cms, sig_obs_scores_cms, poly_models_cms, sig_models_cms = pa_cms.fit_data(medians_cms, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Random permutated data

In [88]:
poly_perm_scores_cms, sig_perm_scores_cms, sig_perm_models_cms=pa_cms.fit_random_data(medians_cms, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Compare obrservable vs permutated data

In [89]:
models_pvalue_cms, scores_table=pa_cms.plot_gof(poly_obs_scores_cms, sig_obs_scores_cms, poly_perm_scores_cms, sig_perm_scores_cms, dist_perm=True)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning

In [90]:
poly_obs_scores_cms = pa_cms.select_significant_models(models_pvalue_cms, poly_obs_scores_cms)

In [91]:
backgrounds_cms = pa_cms.assemble_backgrounds(poly_obs_scores_cms, sig_perm_scores_cms, poly_perm_scores_cms)

In [92]:
p_value_tables_cms = pa_cms.get_p_values(poly_obs_scores_cms, backgrounds_cms)

In [93]:
q_value_tables_cms = pa_cms.get_q_values(p_value_tables_cms, poly_obs_scores_cms)

In [94]:
backgrounds_cms_str = {}
for key in backgrounds_cms.keys():
    backgrounds_cms_str[str(key)] = backgrounds_cms[key]

In [95]:
classification_cms = pa_cms.classify_genes(poly_obs_scores_cms, q_value_tables_cms, 0.2, 0.2)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:782: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_evaluate.dropna(subset=polynomial_columns, how='all', inplace=True)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:791: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_evaluate['continuous'] = only_continuous.iloc[:,0]


In [96]:
for key in classification_cms:
    print(key)
    print(len(classification_cms[key]))

sigmoid
0
discarded
1
continuous
1


In [97]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_cms.items() ])).to_csv(f'{pa_cms.output}/classification_cms.csv')

<ipython-input-97-98d668316869>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_cms.items() ])).to_csv(f'{pa_cms.output}/classification_cms.csv')


# MOLECULAR FEATURES PROFILING

In [98]:
# Create workflow class, specifying the path to the SETTINGS.ini file
pa_mf = ProfileAnalysis('../../../../docker/analysis/molecular_features')

Project has been created!


/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:154: UserWarning: Folder input_data already exist...skipping
  warnings.warn("Folder input_data already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:160: UserWarning: Folder data_raw already exist...skipping
  warnings.warn("Folder data_raw already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:166: UserWarning: Folder data_clinical already exist...skipping
  warnings.warn("Folder data_clinical already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:172: UserWarning: Folder sample_by_section already exist...skipping
  warnings.warn("Folder sample_by_section alr

## Calculate median value for each colon section

In [99]:
medians_mf, mad_tr = pa_mf.median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


In [100]:
medians_mf.drop('Splenic_Flexure', axis=1, inplace=True)

In [101]:
mad_mf = medians_mf

## Fit Observables

In [102]:
scores_mf, poly_obs_scores_mf, sig_obs_scores_mf, poly_models_mf, sig_models_mf = pa_mf.fit_data(medians_mf, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Random permutated data

In [103]:
poly_perm_scores_mf, sig_perm_scores_mf, sig_perm_models_mf = pa_mf.fit_random_data(medians_mf, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Compare obrservable vs permutated data

In [104]:
models_pvalue_mf, scores_table = pa_mf.plot_gof(poly_obs_scores_mf, sig_obs_scores_mf, poly_perm_scores_mf, sig_perm_scores_mf, dist_perm=True)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/

In [105]:
models_pvalue_mf

{'sigmoidal': [720.5, 0.0028803815027847344, 0.8101906026524819],
 1: [880.0, 7.798628094971606e-05, 0.539685665677024],
 2: [848.0, 0.0010332093031969363, 0.6782939398023872],
 3: [796.0, 0.007514784172967473, 0.8607285414561264]}

In [106]:
poly_obs_scores_mf = pa_mf.select_significant_models(models_pvalue_mf, poly_obs_scores_mf)

In [107]:
backgrounds_mf = pa_mf.assemble_backgrounds(poly_obs_scores_mf, sig_perm_scores_mf, poly_perm_scores_mf)

In [108]:
p_value_tables_mf = pa_mf.get_p_values(poly_obs_scores_mf, backgrounds_mf)

In [109]:
q_value_tables_mf = pa_mf.get_q_values(p_value_tables_mf, poly_obs_scores_mf)

In [110]:
backgrounds_mf_str = {}
for key in backgrounds_mf.keys():
    backgrounds_mf_str[str(key)] = backgrounds_mf[key]

In [111]:
classification_mf = pa_mf.classify_genes(poly_obs_scores_mf, q_value_tables_mf, 0.2,0.1)

In [112]:
for key in classification_mf:
    print(key)
    print(len(classification_mf[key]))

sigmoid
2
discarded
0
continuous
1


In [113]:
classification_mf

{'sigmoid': ['MSI', 'CIN'], 'discarded': [], 'continuous': ['CIMP']}

# ANEUPLOIDY PROFILING

## Duplication

In [114]:
# Create workflow class, specifying the path to the SETTINGS.ini file
pa_duplication = ProfileAnalysis('../../../../docker/analysis/aneuploidy/duplication')

Project has been created!


/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:154: UserWarning: Folder input_data already exist...skipping
  warnings.warn("Folder input_data already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:160: UserWarning: Folder data_raw already exist...skipping
  warnings.warn("Folder data_raw already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:166: UserWarning: Folder data_clinical already exist...skipping
  warnings.warn("Folder data_clinical already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:172: UserWarning: Folder sample_by_section already exist...skipping
  warnings.warn("Folder sample_by_section alr

## Assign each sample in clinical data file to a colon section

In [115]:
pa_duplication.create_samples_to_sections_table()

## Calculate median value for each colon section

In [116]:
medians_dup, mad_dup = pa_duplication.median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


In [117]:
#remove samples with more than 4 section containing zeroes
medians_dup=medians_dup[(medians_dup==0).sum(axis=1)<4]

## Fit Observables

In [118]:
scores_dup, poly_obs_scores_dup, sig_obs_scores_dup, poly_models_dup, sig_models_dup = pa_duplication.fit_data(medians_dup, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Random permutated data

In [119]:
poly_perm_scores_dup, sig_perm_scores_dup, sig_perm_models_dup=pa_duplication.fit_random_data(medians_dup, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Compare obrservable vs permutated data

In [120]:
models_pvalue_dup, scores_table=pa_duplication.plot_gof(poly_obs_scores_dup, sig_obs_scores_dup, poly_perm_scores_dup, sig_perm_scores_dup, dist_perm=True)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/

In [121]:
models_pvalue_dup

{'sigmoidal': [47737.0, 0.10835702996192376, 0.750608571706694],
 1: [61008.0, 0.030935250227867372, 0.4801361827868336],
 2: [56470.0, 0.1578712229901772, 0.6824173967685947],
 3: [52809.0, 0.37971178037906733, 0.8340157482725975]}

In [122]:
poly_obs_scores_dup = pa_duplication.select_significant_models(models_pvalue_dup, poly_obs_scores_dup)

In [123]:
backgrounds_dup = pa_duplication.assemble_backgrounds(poly_obs_scores_dup, sig_perm_scores_dup, poly_perm_scores_dup)

In [124]:
p_value_tables_dup = pa_duplication.get_p_values(poly_obs_scores_dup, backgrounds_dup)

In [125]:
q_value_tables_dup = pa_duplication.get_q_values(p_value_tables_dup, poly_obs_scores_dup)

In [126]:
backgrounds_dup_str = {}
for key in backgrounds_dup.keys():
    backgrounds_dup_str[str(key)] = backgrounds_dup[key]

In [127]:
classification_dup = pa_duplication.classify_genes(poly_obs_scores_dup, q_value_tables_dup,0.2, 0.1)

In [128]:
for key in classification_dup:
    print(key)
    print(len(classification_dup[key]))

continuous
8


In [129]:
classification_dup

{'continuous': ['3p', '3q', '6p', '6q', '8p', '12q', '13q', '20q']}

In [130]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_dup.items() ])).to_csv(f'{pa_duplication.output}/results_dup.csv')

## Deletion

In [131]:
# Create workflow class, specifying the path to the SETTINGS.ini file
pa_deletion = ProfileAnalysis('../../../../docker/analysis/aneuploidy/deletion')

Project has been created!


/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:154: UserWarning: Folder input_data already exist...skipping
  warnings.warn("Folder input_data already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:160: UserWarning: Folder data_raw already exist...skipping
  warnings.warn("Folder data_raw already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:166: UserWarning: Folder data_clinical already exist...skipping
  warnings.warn("Folder data_clinical already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:172: UserWarning: Folder sample_by_section already exist...skipping
  warnings.warn("Folder sample_by_section alr

## Assign each sample in clinical data file to a colon section

In [132]:
pa_deletion.create_samples_to_sections_table()

## Calculate median value for each colon section

In [133]:
medians_del, mad_del = pa_deletion.median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


In [134]:
medians_del = medians_del[(medians_del==0).sum(axis=1)<4]

## Fit Observables

In [135]:
scores_del, poly_obs_scores_del, sig_obs_scores_del, poly_models_del, sig_models_del = pa_deletion.fit_data(medians_del, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Random permutated data

In [136]:
poly_perm_scores_del, sig_perm_scores_del, sig_perm_models_del = pa_deletion.fit_random_data(medians_del, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Compare obrservable vs permutated data

In [137]:
models_pvalue_del, scores_table = pa_deletion.plot_gof(poly_obs_scores_del, sig_obs_scores_del, poly_perm_scores_del, sig_perm_scores_del, dist_perm=True)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/Documenti/colon_paper/git/profiler/

In [138]:
models_pvalue_del

{'sigmoidal': [66465.0, 0.004735342023734476, 0.7640185834905403],
 1: [89623.0, 3.737939899787586e-05, 0.5306604958838856],
 2: [74684.0, 0.057399623664145645, 0.7043890970298197],
 3: [64556.0, 0.5155594862177509, 0.8308486036425703]}

In [139]:
poly_obs_scores_del = pa_deletion.select_significant_models(models_pvalue_del, poly_obs_scores_del)

In [140]:
backgrounds_del = pa_deletion.assemble_backgrounds(poly_obs_scores_del, sig_perm_scores_del, poly_perm_scores_del)

In [141]:
p_value_tables_del = pa_deletion.get_p_values(poly_obs_scores_del, backgrounds_del)

In [142]:
q_value_tables_del = pa_deletion.get_q_values(p_value_tables_del, poly_obs_scores_del)

In [143]:
backgrounds_del_str = {}
for key in backgrounds_del.keys():
    backgrounds_del_str[str(key)] = backgrounds_del[key]

In [144]:
classification_del = pa_deletion.classify_genes(poly_obs_scores_del, q_value_tables_del,0.2, 0.1)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:782: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_evaluate.dropna(subset=polynomial_columns, how='all', inplace=True)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:791: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_evaluate['continuous'] = only_continuous.iloc[:,0]


In [145]:
classification_del

{'sigmoid': ['17p', '20p', '1p', '9q', '15q'],
 'discarded': [],
 'continuous': ['4p',
  '4q',
  '5q',
  '10q',
  '11p',
  '11q',
  '18q',
  '6q',
  '8p',
  '14q',
  '18p']}

In [146]:
for key in classification_del:
    print(key)
    print(len(classification_del[key]))

sigmoid
5
discarded
0
continuous
11


In [147]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_del.items() ])).to_csv(f'{pa_deletion.output}/classification_del.csv')

<ipython-input-147-d70f42999164>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_del.items() ])).to_csv(f'{pa_deletion.output}/classification_del.csv')


# SIGNATURES PROFILING

In [148]:
# Create workflow class, specifying the path to the SETTINGS.ini file
pa_sig = ProfileAnalysis('../../../../docker/analysis/signatures')

Project has been created!


/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:154: UserWarning: Folder input_data already exist...skipping
  warnings.warn("Folder input_data already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:160: UserWarning: Folder data_raw already exist...skipping
  warnings.warn("Folder data_raw already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:166: UserWarning: Folder data_clinical already exist...skipping
  warnings.warn("Folder data_clinical already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:172: UserWarning: Folder sample_by_section already exist...skipping
  warnings.warn("Folder sample_by_section alr

## Assign each sample in clinical data file to a colon section

In [149]:
pa_sig.create_samples_to_sections_table()

## Calculate median value for each colon section

In [150]:
medians_sig, mad_sig = pa_sig.median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


In [151]:
medians_sig = medians_sig[(medians_sig==0).sum(axis=1) < 5]

## Fit Observables

In [152]:
scores_sig, poly_obs_scores_sig, sig_obs_scores_sig, poly_models_sig, sig_models_sig = pa_sig.fit_data(medians_sig, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Random permutated data

In [153]:
poly_perm_scores_sig, sig_perm_scores_sig, sig_perm_models_sig=pa_sig.fit_random_data(medians_sig, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Compare obrservable vs permutated data

In [154]:
models_pvalue_sig, scores_table=pa_sig.plot_gof(poly_obs_scores_sig, sig_obs_scores_sig, poly_perm_scores_sig, sig_perm_scores_sig, dist_perm=True)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/

In [155]:
poly_obs_scores_sig = pa_sig.select_significant_models(models_pvalue_sig, poly_obs_scores_sig)

In [156]:
backgrounds_sig = pa_sig.assemble_backgrounds(poly_obs_scores_sig, sig_perm_scores_sig, poly_perm_scores_sig)

In [157]:
p_value_tables_sig = pa_sig.get_p_values(poly_obs_scores_sig, backgrounds_sig)

In [158]:
q_value_tables_sig = pa_sig.get_q_values(p_value_tables_sig, poly_obs_scores_sig)

In [159]:
backgrounds_sig_str = {}
for key in backgrounds_sig.keys():
    backgrounds_sig_str[str(key)] = backgrounds_sig[key]

In [160]:
classification_sig = pa_sig.classify_genes(poly_obs_scores_sig, q_value_tables_sig, 0.2, 0.1)

In [161]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_sig.items() ])).to_csv(f'{pa_sig.output}/classification_sig.csv')

<ipython-input-161-c3b54b7e813b>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_sig.items() ])).to_csv(f'{pa_sig.output}/classification_sig.csv')


In [162]:
classification_sig

{'sigmoid': ['SBS1'], 'discarded': [], 'continuous': ['SBS5']}

In [163]:
for key in classification_sig:
    print(key)
    print(len(classification_sig[key]))

sigmoid
1
discarded
0
continuous
1


In [164]:
from scipy.stats import mannwhitneyu, beta, median_abs_deviation, norm
def calculate_median_by_section_numeric(measurements):
    """
    Calculate samples median and mad values for each section.
    Genes with a % of zero measurements above 'sample_0_t'
    (defined in settings) are discarded from the analysis.

    Parameters
    ----------
    measurements : DataFrame
        table containing measurements for each sample

    Returns
    -------
    medians_df : DataFrame
        DataFrame with median values per section. If the number of zero
        measurements is above 'sample_t_0' DataFrame will be empty.

    mad_df : DataFrame
        DataFrame with MAD for eah section.
    """
    medians_df = pd.DataFrame()
    mad_df = pd.DataFrame()
    for index, feature in measurements.iterrows():
        if int(feature.isin([0]).sum())/len(feature) <= pa_mic.sample_0_t:
            for section in pa_mic.sections:
                values = feature[pa_mic.samples2sections[section]]
                median = np.mean(values)
                mad = median_abs_deviation(values)
                medians_df.loc[index, section] = median
                mad_df.loc[index, section] = mad
    if pa_mic.medians_nan == 'drop':
        medians_df.dropna(inplace=True)
    mad_df = mad_df.loc[medians_df.index]
    return medians_df, mad_df

def median_by_section():
    """
    Wrapper of (calculate_median_by_section).Calculate gene expression
    median and median absolute deviation (MAD) values for each section.

    Parameters
    ----------

    Returns
    -------
    medians_df : DataFrame
        DataFrame with median values per section. If the number of zero
        measurment is above 'sample_t_0' DataFrame will be empty.

    mad_df : DataFrame
        DataFrame with MAD for eah section.
    """
    load_results_median = pa_mic.check_step_completion('/'.join([pa_mic.sample_by_section, 'median_by_sections.csv']))
    load_results_mad = pa_mic.check_step_completion('/'.join([pa_mic.sample_by_section, 'mad_by_sections.csv']))
    if load_results_median.empty or load_results_mad.empty:
        medians = pd.DataFrame()
        mad = pd.DataFrame()
        if pa_mic.data_type == 'numeric':
            calc_res = Parallel(n_jobs=pa_mic.cores)(delayed(calculate_median_by_section_numeric)(group) for i, group in pa_mic.data_table.groupby(np.arange(len(pa_mic.data_table)) // pa_mic.cores))
        elif pa_mic.data_type == 'binary':
            calc_res = Parallel(n_jobs=pa_mic.cores)(delayed(calculate_median_by_section_binary)(group) for i, group in pa_mic.data_table.groupby(np.arange(len(pa_mic.data_table)) // pa_mic.cores))

        for res in calc_res:
            if not res[0].empty:
                medians = pd.concat([medians, res[0]])
                # medians.loc[res[0].index]=res[0]
                # mad = mad.append(res[1])
                mad = pd.concat([mad, res[1]])
        medians.to_csv('/'.join([pa_mic.sample_by_section, 'median_by_sections.csv']))
        mad.to_csv('/'.join([pa_mic.sample_by_section, 'mad_by_sections.csv']))
    else:
        medians = load_results_median
        medians.columns.values[0] = pa_mic.index_col
        medians.set_index(pa_mic.index_col, inplace=True)
        mad = load_results_mad

    return medians, mad

# MICROBIOME PROFILING

In [165]:
# Create workflow class, specifying the path to the SETTINGS.ini file
pa_mic = ProfileAnalysis('../../../../docker/analysis/microbiome')

Project has been created!


/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:154: UserWarning: Folder input_data already exist...skipping
  warnings.warn("Folder input_data already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:160: UserWarning: Folder data_raw already exist...skipping
  warnings.warn("Folder data_raw already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:166: UserWarning: Folder data_clinical already exist...skipping
  warnings.warn("Folder data_clinical already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:172: UserWarning: Folder sample_by_section already exist...skipping
  warnings.warn("Folder sample_by_section alr

## Assign each sample in clinical data file to a colon section

In [166]:
pa_mic.create_samples_to_sections_table()

## Calculate median value for each colon section

In [167]:
medians_mic, mad_mic = median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


In [168]:
gmrepo = pd.read_csv('/home/ieo5417/Documenti/colon_paper/docker/analysis/microbiome/GMREPO_relative_abundance_of_all_species_genus_in_all_phenotypes_summary.tsv', skiprows=2, sep='\t')

In [169]:
gmrepo = gmrepo[gmrepo['samples in which current taxon is found']>100]

In [170]:
gmspecies = gmrepo[gmrepo['taxonomic rank']=='species']
species_to_keep = list(gmspecies['NCBI taxon ID'].unique())

In [171]:
id_conversion_table=pd.read_csv('../../../../docker/analysis/microbiome/misc/all_species.txt', sep='\t')
id_conversion_table.set_index('tax_id', inplace=True)
species_table = pd.DataFrame()
not_found = []
for index, row in medians_mic.iterrows():
    if index in id_conversion_table.index:
        species_table.loc[index,'species'] = id_conversion_table.loc[index,'taxon_name']
        species_table.loc[index,'genus'] = id_conversion_table.loc[index,'taxon_name'].split('_')[0]
    else:
        not_found.append(index)

In [172]:
mask = [id_ in species_to_keep for id_ in medians_mic.index]
medians_mic = medians_mic.loc[mask]

In [173]:
medians_mic = medians_mic[(medians_mic == 0).sum(axis=1)<4]

## Fit Observables

In [174]:
scores_mic, poly_obs_scores_mic, sig_obs_scores_mic, poly_models_mic, sig_models_mic = pa_mic.fit_data(medians_mic, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Random permutated data

In [175]:
poly_perm_scores_mic, sig_perm_scores_mic, sig_perm_models_mic = pa_mic.fit_random_data(medians_mic, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Compare obrservable vs permutated data

In [176]:
models_pvalue_mic, scores_table = pa_mic.plot_gof(poly_obs_scores_mic, sig_obs_scores_mic, poly_perm_scores_mic, sig_perm_scores_mic, dist_perm=True)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:327: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:333: RuntimeWarning: invalid value

In [177]:
models_pvalue_mic

{'sigmoidal': [19228722.0, 0.9999996512819851, 0.9792131759121212],
 1: [28120294.5, 0.002222588511645301, 0.4368332248476715],
 2: [26945527.5, 0.21674933418183284, 0.7059207453619665],
 3: [26966507.0, 0.20609842896875935, 0.8804665812832226]}

In [178]:
poly_obs_scores_mic = pa_mic.select_significant_models(models_pvalue_mic, poly_obs_scores_mic)

In [179]:
col=[]
for column in poly_obs_scores_mic.columns:
    if column != 'sigmoidal': 
        col.append(int(column))
    else:
        col.append('sigmoidal')
poly_obs_scores_mic.columns = col

In [180]:
backgrounds_mic = pa_mic.assemble_backgrounds(poly_obs_scores_mic, sig_perm_scores_mic, poly_perm_scores_mic)

In [181]:
p_value_tables_mic = pa_mic.get_p_values(poly_obs_scores_mic, backgrounds_mic)

In [182]:
q_value_tables_mic = pa_mic.get_q_values(p_value_tables_mic, poly_obs_scores_mic)

In [183]:
q_value_tables_mic[q_value_tables_mic[1]<=0.2].index

Float64Index([  33038.0,     837.0,    1680.0,  658082.0,  398037.0,  301301.0,
              1055487.0,  309120.0,  410072.0,   53443.0,   87883.0,  351091.0,
               157688.0,  452623.0,  552810.0,  259537.0,   37928.0],
             dtype='float64')

In [184]:
backgrounds_mic_str = {}
for key in backgrounds_mic.keys():
    backgrounds_mic_str[str(key)] = backgrounds_mic[key]

In [185]:
classification_mic = pa_mic.classify_genes(poly_obs_scores_mic, q_value_tables_mic, 0.2, 0.1)

In [186]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_mic.items() ])).to_csv(f'{pa_mic.output}/classification_mic.csv')

# IMMUNE CELLS PROFILING

In [187]:
# Create workflow class, specifying the path to the SETTINGS.ini file
pa_imm = ProfileAnalysis('../../../../docker/analysis/immune_cells')

Project has been created!


/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:154: UserWarning: Folder input_data already exist...skipping
  warnings.warn("Folder input_data already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:160: UserWarning: Folder data_raw already exist...skipping
  warnings.warn("Folder data_raw already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:166: UserWarning: Folder data_clinical already exist...skipping
  warnings.warn("Folder data_clinical already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:172: UserWarning: Folder sample_by_section already exist...skipping
  warnings.warn("Folder sample_by_section alr

## Assign each sample in clinical data file to a colon section

In [188]:
pa_imm.create_samples_to_sections_table()

## Calculate median value for each colon section

In [189]:
medians_imm, mad_imm = pa_imm.median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Observables

In [190]:
scores_imm, poly_obs_scores_imm, sig_obs_scores_imm, poly_models_imm, sig_models_imm = pa_imm.fit_data(medians_imm, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Random permutated data

In [191]:
poly_perm_scores_imm, sig_perm_scores_imm, sig_perm_models_imm=pa_imm.fit_random_data(medians_imm, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Compare obrservable vs permutated data

In [192]:
models_pvalue_imm, scores_table=pa_imm.plot_gof(poly_obs_scores_imm, sig_obs_scores_imm, poly_perm_scores_imm, sig_perm_scores_imm, dist_perm=True)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler

In [193]:
models_pvalue_imm

{'sigmoidal': [3570.0, 0.2433828513924995, 0.8084061471488586],
 1: [5540.0, 0.028683752848333424, 0.4930886981230882],
 2: [5253.0, 0.062476969704019975, 0.7030799177700063],
 3: [5198.0, 0.07157899842862023, 0.8405262850281643]}

In [194]:
poly_obs_scores_imm = pa_imm.select_significant_models(models_pvalue_imm, poly_obs_scores_imm)

In [195]:
backgrounds_imm = pa_imm.assemble_backgrounds(poly_obs_scores_imm, sig_perm_scores_imm, poly_perm_scores_imm)

In [196]:
p_value_tables_imm = pa_imm.get_p_values(poly_obs_scores_imm, backgrounds_imm)

In [197]:
q_value_tables_imm = pa_imm.get_q_values(p_value_tables_imm, poly_obs_scores_imm)

In [198]:
q_value_tables_imm[q_value_tables_imm[1]<0.2]

,1
T cells CD8,0.016404
T cells CD4 memory resting,0.196583
T cells follicular helper,0.017924
Macrophages M0,0.011191


In [199]:
backgrounds_imm_str = {}
for key in backgrounds_imm.keys():
    backgrounds_imm_str[str(key)] = backgrounds_imm[key]

In [200]:
classification_imm = pa_imm.classify_genes(poly_obs_scores_imm, q_value_tables_imm, 0.2, 0.1)

In [201]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_imm.items() ])).to_csv(f'{pa_imm.output}/classification_imm.csv')

In [202]:
for key in classification_imm:
    print(key)
    print(len(classification_imm[key]))

continuous
4


# METABOLITES PROFILING

In [203]:
pa_met=ProfileAnalysis('../../../../docker/analysis/metabolites')

Project has been created!


/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:154: UserWarning: Folder input_data already exist...skipping
  warnings.warn("Folder input_data already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:160: UserWarning: Folder data_raw already exist...skipping
  warnings.warn("Folder data_raw already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:166: UserWarning: Folder data_clinical already exist...skipping
  warnings.warn("Folder data_clinical already exist...skipping",
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:172: UserWarning: Folder sample_by_section already exist...skipping
  warnings.warn("Folder sample_by_section alr

## Assign each sample in clinical data file to a colon section

In [204]:
pa_met.create_samples_to_sections_table()

# Figure 2C

In [205]:
pa_met.plot_sample_distribution(title='IEO sample distribution')

# Figure 2B

In [206]:
import numpy as np
labels = ['Sigmoid', 'Continuous']
metabolites = [0, 10]
microbiome = [0, 17]
immune = [0, 4]
sbs=[1, 1]

x = np.arange(len(labels))  # the label locations
width = 0.15  # the width of the bars

fig, ax = plt.subplots(figsize=(10,10))
rects3 = ax.bar(x - width/2, immune, width, label='Immune cells', edgecolor='k')
rects1 = ax.bar(x+width/2, metabolites, width, label='Metabolites', edgecolor='k')
rects2 = ax.bar(x+width*1.5, microbiome, width, label='Microbes', edgecolor='k')
# rects4 = ax.bar(x - width*1.5, sbs, width, label='Mutational Signatures', edgecolor='k')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Number of features')
ax.set_xticks(x, labels)
ax.legend(loc='upper left')

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)
ax.bar_label(rects3, padding=3)
# ax.bar_label(rects4, padding=3)

fig.tight_layout()
plt.show()

<ipython-input-206-be7063c46826>:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Calculate median value for each colon section

In [207]:
medians_met, mad_met=pa_met.median_by_section()

This step has already been executed...loading results...
This step has already been executed...loading results...


In [208]:
medians_met = medians_met[medians_met.isna().sum(axis=1)<2]

In [209]:
medians_met

,Cecum,Ascending colon,Hepatic flexure of colon,Splenic flexure of colon,Descending colon,Sigmoid colon,Rectosigmoid junction,"Rectum, NOS"
metabolites,,,,,,,,
49,0.916616,1.137239,2.802380,0.648456,1.269340,NaN,2.798767,4.794570
50,1.272790,1.911105,2.946900,1.279519,5.783875,2.953145,3.947325,3.587472
55,2.655592,2.484064,7.122336,3.048335,10.699613,4.873210,6.457502,5.667234
93,1.142999,0.749726,1.930918,0.817009,2.082914,1.826542,2.084325,1.009596
112,1.324899,1.132940,2.102044,1.212122,2.663467,1.078540,3.402928,1.740025
...,...,...,...,...,...,...,...,...
100015878,1.179367,0.614141,0.331211,1.344607,1.573660,0.970451,1.608553,0.384793
100015962,2.356501,0.912239,2.892607,1.451085,3.214546,1.910643,1.452099,1.639024
100019915,0.785966,0.545137,3.287475,1.295955,0.620813,2.648316,3.654804,1.583657


In [210]:
medians_met.fillna(0, inplace=True)

In [211]:
medians_met = medians_met.loc[medians_met.median(axis=1)>1]

## Fit Observables

In [212]:
models_scores_met, poly_obs_fit_scores_met, sig_obs_fit_scores_met, poly_models_met, sig_models_met=pa_met.fit_data(medians_met, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Fit Random permutated data

In [213]:
poly_perm_fit_scores_met, sig_perm_fit_scores_met, sig_perm_models_met=pa_met.fit_random_data(medians_met, guess_bounds = True)

This step has already been executed...loading results...
This step has already been executed...loading results...
This step has already been executed...loading results...


## Compare obrservable vs permutated data

In [214]:
models_pvalue_table_met, scores_table=pa_met.plot_gof(poly_obs_fit_scores_met, sig_obs_fit_scores_met, poly_perm_fit_scores_met, sig_perm_fit_scores_met)

/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:667: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/Documenti/colon_paper/git/profiler/notebooks/paper_analysis/../../../../git/profiler/lib/profile_analysis_class.py:638: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:327: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:333: RuntimeWarning: invalid value encountered in accumulate
  pvals_corrected = np.minimum.accumulate(pvals_corrected_raw[::-1])[::-1]
/home/ieo5417/.local/lib/python3.8/site-packages/statsmodels/stats/multitest.py:335: RuntimeWarning

In [215]:
models_pvalue_table_met

{'sigmoidal': [1316598.0, 0.9566746155699346, 0.7988985985265682],
 1: [1611086.0, 0.04124031188292741, 0.4942916822293544],
 2: [1503774.0, 0.4558341021896764, 0.7186487932334056],
 3: [1375318.0, 0.9667386923914669, 0.8365184631833656]}

In [216]:
poly_obs_fit_scores_met = pa_met.select_significant_models(models_pvalue_table_met, poly_obs_fit_scores_met)

In [217]:
backgrounds_met = pa_met.assemble_backgrounds(poly_obs_fit_scores_met, sig_perm_fit_scores_met, poly_perm_fit_scores_met)

In [218]:
p_value_tables_met = pa_met.get_p_values(poly_obs_fit_scores_met, backgrounds_met)

In [219]:
q_value_tables_met = pa_met.get_q_values(p_value_tables_met, poly_obs_fit_scores_met)

In [220]:
backgrounds_met_str = {}
for key in backgrounds_met.keys():
    backgrounds_met_str[str(key)] = backgrounds_met[key]

In [221]:
classification_met = pa_met.classify_genes(poly_obs_fit_scores_met, q_value_tables_met,0.2, 0.1)

In [222]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in classification_met.items() ])).to_csv(f'{pa_met.output}/classification_met.csv')

In [223]:
for key in classification_met:
    print(key)
    print(len(classification_met[key]))

continuous
10


In [224]:
name_conversion = pd.read_csv('../../../../docker/analysis/metabolites/metabolites_id_to_names.csv')
name_dict={}
for index in q_value_tables_met[q_value_tables_met['1']<0.2].index:
    name_dict[index]=name_conversion[name_conversion['CHEM_ID']==index]['CHEMICAL_NAME'].item()
metabolites_name_df=pd.DataFrame(name_dict, index=['metabolite name']).T

In [225]:
metabolites_name_df

,metabolite name
197.0,S-adenosylhomocysteine (SAH)
439.0,stearate (18:0)
507.0,spermine
798.0,adenosine
1087.0,erucate (22:1n9)
1124.0,citrate
100001054.0,butyrylcarnitine (C4)
100001162.0,propionylcarnitine (C3)
100001335.0,eicosenoate (20:1)
100008930.0,oleate/vaccenate (18:1)


In [226]:
import math

def summary_profile_plot(medians, mad, indexes, metabolite_names, poly_models):
    if 'Unnamed: 0' in mad_met.columns:
        mad_met.set_index('Unnamed: 0', inplace=True)
    data=medians.loc[indexes]
    data_err=mad.loc[indexes]
    data.fillna(0, inplace=True)
    
    x = np.linspace(pa_met.x[0], pa_met.x[-1], 1000)
    degree = 1
    y = np.polynomial.polynomial.polyval(x, poly_models[degree][data.index[0]])
    
    n = 0    
    fig = plt.figure(figsize=(15, 15))
    axes=[]
    ax_base = plt.subplot2grid((9,2), (0,0))
    axes.append(ax_base)
    ax_base.errorbar(pa_met.x, data.loc[data.index[0]], yerr=data_err.loc[data.index[0]],
                     label=metabolite_names.loc[data.index[0]].item(),
                    marker='o', markeredgecolor='k', markerfacecolor='red', color='k')
    ax_base.plot(x, y, color='red', ls='-', linewidth=2, alpha=0.7)
    max_lim = math.ceil(data_err.loc[data.index[0]].max()+data.loc[data.index[0]].max())
    ax_base.axis(ymin=0, ymax=max_lim)
    ax_base.set_yticks([0, max_lim]) 
    ax_base.set_yticklabels([str(0), str(max_lim)], fontsize=15)
    ax_base.set_title(metabolite_names.loc[data.index[0]].item(), y=1.0, x=0.01, pad=-17, loc='left', fontsize=12)
    l=1
    h=0
    for index in data.index[1:len(data.index)]:
        degree = 1
        y = np.polynomial.polynomial.polyval(x, poly_models[degree][index])
        ax = plt.subplot2grid((9,2), (h,l), sharex=ax_base)
        axes.append(ax)
        markers, caps, bars = ax.errorbar(pa_met.x, data.loc[index], yerr=data_err.loc[index], label=metabolite_names.loc[index],
                    marker='o', markeredgecolor='k', markerfacecolor='red', color='k')
        ax.plot(x, y, color='red', ls='-', linewidth=2, alpha=0.7)
        ax.set_xticks(pa_met.x)
        ax.set_xticklabels(pa_met.sections4plots, rotation=45, ha='right', fontsize=15)
        max_lim = math.ceil(data_err.loc[index].max()+data.loc[index].max())
        ax.axis(ymin=0, ymax=max_lim)
        ax.set_yticks([0, max_lim])
        ax.set_yticklabels([str(0), str(max_lim)], fontsize=15)
        ax.set_title(metabolite_names.loc[index].item(), y=1.0, x=0.01, pad=-17, loc='left', fontsize=12)
        l = l+1
        if l == 2:
            h = h+1
            l = 0
    fig.subplots_adjust(hspace=0.25)   
    for ax in axes[0:-2]:
        plt.setp(ax.get_xticklabels(), visible=False)    
    plt.savefig('/'.join([pa_transcriptome.figures, 'continuous_metabolites.svg']), format='svg')

# Correlation metabolites/microbes

In [227]:
metabolite_names = pd.read_csv('../../../../docker/analysis/metabolites/metabolites_id_to_names.csv')

metabolites_medians = medians_met.loc[q_value_tables_met[q_value_tables_met['1']<0.2].index]
metabolites_medians.fillna(0, inplace=True)

for index in metabolites_medians.index:
    metabolites_medians.loc[index, 'name'] = metabolite_names[metabolite_names['CHEM_ID']==index]['CHEMICAL_NAME'].item()
metabolites_medians.set_index('name', inplace=True)

In [228]:
# microbes = pd.read_csv('/home/ieo5417/Documenti/colon_paper/docker/analysis/microbiome/output/continuum.csv')
# microbes.set_index('Unnamed: 0', inplace=True)
microbes_names = pd.read_csv('../../../../docker/analysis/microbiome/misc/all_species.txt', sep='\t')

# microbes_medians = pd.read_csv('/home/ieo5417/Documenti/colon_paper/docker/analysis/microbiome/sample_by_section_1/median_by_sections.csv')
# microbes_medians.set_index('Unnamed: 0', inplace=True)
microbes_medians = medians_mic.loc[q_value_tables_mic[q_value_tables_mic['1']<0.2].index]
microbes_medians.dropna(inplace=True)
for index in microbes_medians.index:
    if index in list(microbes_names['tax_id']):
        microbes_medians.loc[index, 'name'] = microbes_names[microbes_names['tax_id']==index]['taxon_name'].item()
microbes_medians.dropna(inplace=True)
microbes_medians.set_index('name', inplace=True)

In [229]:
i=1
correlations = pd.DataFrame()
corr_mtrx = pd.DataFrame()
p_mtrx = pd.DataFrame()
for index_met, row_met in metabolites_medians.iterrows():
    for index_micro, row_micro in microbes_medians.iterrows():
#         temp=row_met.drop(['Splenic flexure of colon','Descending colon','Rectosigmoid junction'])
        stat,p = scipy.stats.pearsonr(row_met, row_micro)
        correlations.loc[i,'microbe'] = index_micro
        correlations.loc[i,'metabolite'] = index_met
        correlations.loc[i,'pearson'] = stat
        correlations.loc[i,'p-pearson'] = p
        corr_mtrx.loc[index_met,index_micro] = stat
        p_mtrx.loc[index_met,index_micro] = p
        stat,p = scipy.stats.spearmanr(row_met, row_micro)
        correlations.loc[i,'spearman'] = stat
        correlations.loc[i,'p-spearman'] = p
        correlations.loc[i,'genus'] = index_micro.split('_')[0]
        i=i+1
correlations['q-value'] = fdrcorrection(correlations['p-pearson'])[1]

In [230]:
correlations

,microbe,metabolite,pearson,p-pearson,spearman,p-spearman,genus,q-value
1,[Ruminococcus]_gnavus,S-adenosylhomocysteine (SAH),-0.612571,0.106413,-0.690476,0.057990,[Ruminococcus],0.190423
2,Porphyromonas_gingivalis,S-adenosylhomocysteine (SAH),-0.822500,0.012186,-0.910196,0.001691,Porphyromonas,0.098982
3,Bifidobacterium_adolescentis,S-adenosylhomocysteine (SAH),0.745173,0.033865,0.428571,0.289403,Bifidobacterium,0.118294
4,Lachnospiraceae_bacterium_2_1_58FAA,S-adenosylhomocysteine (SAH),-0.574122,0.136680,-0.666667,0.070988,Lachnospiraceae,0.208073
5,Segetibacter_koreensis,S-adenosylhomocysteine (SAH),-0.819371,0.012809,-0.952381,0.000260,Segetibacter,0.098982
...,...,...,...,...,...,...,...,...
166,Leptotrichia_hofstadii,oleate/vaccenate (18:1),-0.729847,0.039844,-0.595238,0.119530,Leptotrichia,0.118294
167,Pseudoclavibacter_soli,oleate/vaccenate (18:1),0.402974,0.322237,0.619048,0.101733,Pseudoclavibacter,0.344530
168,Dehalogenimonas_lykanthroporepellens,oleate/vaccenate (18:1),0.496226,0.211031,0.414758,0.306912,Dehalogenimonas,0.256252
169,Dechloromonas_aromatica,oleate/vaccenate (18:1),0.606343,0.111027,0.690476,0.057990,Dechloromonas,0.192597


In [231]:
summary_profile_plot(medians_met, mad_met, metabolites_name_df.index, metabolites_name_df, poly_models_met)

# Figure 2D

In [232]:
profiling_results = {}
unclassified_tr=len(medians_tr)-len(classification_tr['continuous'])-len(classification_tr['sigmoid'])
profiling_results['RNA-seq'] = [len(classification_tr['continuous']),len(classification_tr['sigmoid']),unclassified_tr]

unclassified_meth=len(medians_meth)-len(classification_meth['continuous'])-len(classification_meth['sigmoid'])
profiling_results['Methylation'] = [len(classification_meth['continuous']),len(classification_meth['sigmoid']),unclassified_meth]

unclassified_mut=len(medians_mut)-len(classification_mut['continuous'])-len(classification_mut['sigmoid'])
profiling_results['SNV'] = [len(classification_mut['continuous']),len(classification_mut['sigmoid']),unclassified_mut]

unclassified_dup=len(medians_dup)-len(q_value_tables_dup[q_value_tables_dup['1']<0.2])-0
unclassified_del=len(medians_del)-len(classification_del['continuous'])-len(classification_del['sigmoid'])
profiling_results['Aneuploidy'] = [len(q_value_tables_dup[q_value_tables_dup['1']<0.2])+len(classification_del['continuous']),
                                   0+len(classification_del['sigmoid']),
                                   unclassified_dup+unclassified_del]

unclassified_cms=len(medians_cms)-len(classification_cms['continuous'])-len(classification_cms['sigmoid'])
profiling_results['CMS'] = [len(classification_cms['continuous']),len(classification_cms['sigmoid']),unclassified_cms]

unclassified_mf=len(medians_mf)-len(classification_mf['continuous'])-len(classification_mf['sigmoid'])
profiling_results['Molecular Subtypes'] = [len(classification_mf['continuous']),len(classification_mf['sigmoid']),unclassified_mf]

# Figure 1C

In [233]:
from textwrap import wrap

cm = 1/2.54  # centimeters in inches
l=0
h=0
wedgeprops = {'width':0.3, 'edgecolor':'black', 'linewidth':2}
colors = ['#0b55cc', '#f1c232','#808080']
fig = plt.figure(figsize=(13*cm, 6*cm))
names = ['Continuous', 'Sigmoid', 'Unclassified']

for key in profiling_results:
    ax = plt.subplot2grid((2,5), (h,l))
    cont = profiling_results[key][0]
    sig = profiling_results[key][1]
    unc = profiling_results[key][2]
    wedges, texts= ax.pie([cont, sig, unc], wedgeprops=wedgeprops, colors=colors,
                                  textprops=dict(color="k"),startangle=180)

    ax.set_title('\n'.join(wrap(key,11)), fontsize=10, loc='center', pad=1)
    l = l+1
    if l == 3:
        h = h+1
        l = 0
ax.legend(wedges, names,
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1), prop={'size': 10}, framealpha=0)
plt.savefig('/'.join([pa_mic.figures, 'Feature_Summary.svg']), format='svg')

In [234]:
'/'.join([pa_mic.figures, 'Feature_Summary.svg'])

'../../../../docker/analysis/microbiome/figures/Feature_Summary.svg'

# NETWORK ANALYSIS

In [235]:
biopax_path = '../../../../docker/analysis/network_analysis_COAD_READ/biopax'
graphml_path = '../../../../docker/analysis/network_analysis_COAD_READ/graphml'
metabolites_path = '../../../../docker/analysis/network_analysis_COAD_READ/input_data/input_metabolites.csv'
genes_path = '../../../../docker/analysis/transcriptome/output/continuous_genes_list.txt'

NSA=NetworkSearchAnalysis(graphml_path, biopax_path, metabolites_path, genes_path)

In [236]:
print('mapping genes to pathway...')
NSA.gene2pathways, NSA.uniprot2ensmbl, NSA.reactome2pathway_names = NSA.map_genes2pathways(NSA.gene_list, scopes='ensemblgene')
print('done!')
print('loading pathways...')
pathways_to_load = [x for x in set(sum(NSA.gene2pathways.values(), []))]
NSA.pathways_graphs, NSA.directed_graph = NSA.load_pathways(pathways_to_load)
NSA.reactome2chebi, NSA.reactome2nodeid_chebi = NSA.map_chebi_to_pathway(NSA.pathways_graphs)
print('done!')
print('select metabolites to analyze')
NSA.target_chebis2pathways = NSA.findTargetChebiInPathways(NSA.reactome2chebi, NSA.metabolites_dict)
print('done!')
print('get metabolites networks ids...')
NSA.ids_to_analyze = NSA.mapIdToAnalyze(NSA.target_chebis2pathways, NSA.reactome2nodeid_chebi)
print('done!')
print('create random test statistics distributions...')
NSA.random_t_stats = NSA.calculate_random_t_stats(NSA.ids_to_analyze, NSA.pathways_graphs)
print('done!')
print('calculate test statistics of observables...')
NSA.pathways_distances_uniprot, NSA.pathway_test_statistics = NSA.calculate_t_stats(NSA.ids_to_analyze, NSA.pathways_graphs)
print('done!')
print('fitting random test statistics distributions...')
NSA.fitting = NSA.fit_distribution(NSA.random_t_stats)
print('done!')
print('get valid genes...')
NSA.pathway_dist, NSA.pdfs = NSA.get_pathway_dist(NSA.random_t_stats, NSA.fitting)
print('done!')
print('calculate p-values ...')
NSA.statistics_df, NSA.p_tresh = NSA.get_p_value(NSA.pathway_dist, NSA.pathway_test_statistics)
print('done!')
print('generate results...')
NSA.final_table = NSA.make_results_table(NSA.statistics_df, NSA.pathways_distances_uniprot)
print('finished!')

mapping genes to pathway...
querying 1-846...done.
Finished.
8 input query terms found dup hits:
	[('ENSG00000088298', 2), ('ENSG00000171714', 2), ('ENSG00000204577', 4), ('ENSG00000223403', 2), ('E
7 input query terms found no hit:
	['ENSG00000225163', 'ENSG00000256050', 'ENSG00000272567', 'ENSG00000273837', 'ENSG00000256045', 'ENS
done!
loading pathways...
pathway R-HSA-9827857 not found!
pathway R-HSA-9824439 not found!
pathway R-HSA-9832991 not found!
pathway R-HSA-9772755 not found!
pathway R-HSA-9762293 not found!
pathway R-HSA-9824446 not found!
pathway R-HSA-9833482 not found!
pathway R-HSA-9759476 not found!
pathway R-HSA-9824272 not found!
pathway R-HSA-9821002 not found!
pathway R-HSA-9764260 not found!
pathway R-HSA-9818030 not found!
pathway R-HSA-9759475 not found!
pathway R-HSA-9820841 not found!
pathway R-HSA-9824443 not found!
pathway R-HSA-9823730 not found!
pathway R-HSA-9820865 not found!
pathway R-HSA-9732724 not found!
pathway R-HSA-9840310 not found!
pathway R-HS

/home/ieo5417/.local/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2789: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu


done!
get valid genes...
done!
calculate p-values ...
done!
generate results...
finished!


In [237]:
NSA.final_table.to_csv('../../../../docker/analysis/network_analysis_COAD_READ/pathway_analysis_continuous_genes.csv')

In [238]:
for index, row in NSA.final_table.iterrows():
    temp_l = ''
    for element in row['genes_uniprot']:
        if temp_l:
            temp_l =temp_l+f', {element}'
        else:
            temp_l = temp_l+element
    NSA.final_table.loc[index, 'genes_uniprot'] = temp_l

In [239]:
summary_groups = NSA.final_table.groupby(['metabolite','genes_uniprot'])
summary_table = pd.DataFrame()
for name, group in summary_groups:
    counter = collections.Counter(group['genes_uniprot'])
#     print(counter)
    for element in counter:
        if counter[element] > 1:
            rows_to_summarize = group[group['genes_uniprot']==element]
            row_to_keep = rows_to_summarize['q_value'].min()
            summary_table = pd.concat([summary_table, rows_to_summarize[rows_to_summarize['q_value']==row_to_keep]])
        else:
            summary_table = pd.concat([summary_table, group[group['genes_uniprot']==element]])
summary_table.to_csv('../../../../docker/analysis/network_analysis_COAD_READ/SUMMARY_pathway_analysis_continuous_genes.csv')

# END